In [94]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from ideas import intersection_over_union

In [36]:
box_columns = ['Xmin','Ymin','Xmax','Ymax']
box_columns_true = ['Xmin_true','Ymin_true','Xmax_true','Ymax_true']
box_columns_id = box_columns + ['itemId']

In [30]:
te_d = pd.read_csv('test_data.csv')
t_d = pd.read_csv('train_data.csv')
t_a = pd.read_csv('train_answers.csv')
t_s = t_d.merge(t_a, on="itemId")
uids = list(set(t_s['userId']))
ids = list(set(te_d['itemId']))

In [56]:
scaler = MinMaxScaler()
regr = linear_model.LinearRegression()

In [68]:
answer = pd.DataFrame(np.zeros((len(te_d), 5), dtype=int), index=te_d.index, columns=box_columns_id)

In [119]:
answer = pd.DataFrame(np.zeros((len(te_d), 5), dtype=int), index=te_d.index, columns=box_columns_id)
# for uid in uids:
for uid in uids:
    X_train = t_s[t_s['userId'] == uid][box_columns]
    y_train = t_s[t_s['userId'] == uid][box_columns_true]
    X_test = te_d[te_d['userId'] == uid][box_columns_id]    
    if (X_test.shape[0] > 0 and X_train.shape[0] > 0):
        scaler.fit(X_train[box_columns])
        regr.fit(scaler.transform(X_train), scaler.transform(y_train))
        y_pred = scaler.inverse_transform(regr.predict(scaler.transform(X_test[box_columns])))
        y = 0
        for i,r in X_test.iterrows():        
            for ci,cv in enumerate(box_columns):
                answer.iloc[i][cv] = y_pred[y,ci] 
            answer.iloc[i]['itemId'] = r['itemId']
            y = y + 1
for i,r in te_d.iterrows():            
    if answer.iloc[i]['itemId'] == 0:
        answer.iloc[i][box_columns_id] = r[box_columns_id]

In [152]:
a_avg = answer[box_columns].groupby("itemId").mean()
print(a_avg)

              Xmin         Ymin         Xmax         Ymax
itemId                                                   
18      301.500000   702.500000   638.000000   975.500000
19      -69.000000   601.000000   245.000000   832.333333
33       39.666667   444.333333   391.333333   697.666667
62      155.142857   966.571429   537.428571  1262.428571
114      41.333333   760.333333   316.000000   908.333333
146      60.750000   604.500000   712.250000  1182.250000
156      76.000000   799.000000   453.333333  1180.666667
163      74.333333   622.166667   487.000000   939.666667
164      84.333333   315.000000   472.666667   679.000000
179      50.000000   435.666667   467.333333   699.333333
186     112.571429   929.857143   922.142857  1726.000000
215     119.000000   602.333333   616.000000   930.333333
225      19.375000   757.750000    85.500000   791.500000
227     123.200000   753.000000   738.800000  1215.400000
247      49.250000   627.500000   142.250000   704.750000
275      73.00

In [154]:
a_avg.astype('int64').to_csv('submit15.csv', header=False)

In [151]:
max_result = pd.read_csv('submit-max.csv')
max_result = max_result.set_index('itemId')
print(max_result.shape)

(630, 4)


In [153]:
data = a_avg.merge(max_result, on=['itemId'])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
print(data["iou"].mean()) 

0.31358463036980755
